<a href="https://colab.research.google.com/github/sattia7/Credit-Card-Fraud-Detection-ML-project/blob/main/credit_card_fraud_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [184]:
#importing the dependecies

In [3]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import csv

In [4]:
#loading the dataset to pandas dataframe 
credit_card_data = pd.read_csv('/content/creditcard.csv', sep=' ,', names= ["Time"])
#credit_card_data.columns.str.split(',',n=40, expand = True)

credit_card_data.head()


/usr/local/lib/python3.7/dist-packages/pandas/util/_decorators.py:311: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  return func(*args, **kwargs)


,Time
0,"""Time"",""V1"",""V2"",""V3"",""V4"",""V5"",""V6"",""V7"",""V8""..."
1,"0,-1.3598071336738,-0.0727811733098497,2.53634..."
2,"0,1.19185711131486,0.26615071205963,0.16648011..."
3,"1,-1.35835406159823,-1.34016307473609,1.773209..."
4,"1,-0.966271711572087,-0.185226008082898,1.7929..."


In [5]:
#credit_card_data.columns.str.split(',',n=40, expand = True)
#df = pd.DataFrame (credit_card_data, sep=',')
#df = spark.read.format("csv").option("header",true).load('/content/creditcard.csv')

df=credit_card_data['Time'].str.split(',',n=30, expand = True)
new_header = df.iloc[0] #grab the first row for the header
df = df[1:] #take the data less the header row
df.columns = new_header #set the header row as the df header
new_headers = []
for header in df.columns: # data.columns is list of headers
    header = header.strip('"') # Remove the quotes off each header
    new_headers.append(header) # Save the new strings without the quotes

df.columns = new_headers
df.head()
#df.info()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
1,0,-1.3598071336738,-0.0727811733098497,2.53634673796914,1.37815522427443,-0.338320769942518,0.462387777762292,0.239598554061257,0.0986979012610507,0.363786969611213,...,-0.018306777944153,0.277837575558899,-0.110473910188767,0.0669280749146731,0.128539358273528,-0.189114843888824,0.133558376740387,-0.0210530534538215,149.62,"""0"""
2,0,1.19185711131486,0.26615071205963,0.16648011335321,0.448154078460911,0.0600176492822243,-0.0823608088155687,-0.0788029833323113,0.0851016549148104,-0.255425128109186,...,-0.225775248033138,-0.638671952771851,0.101288021253234,-0.339846475529127,0.167170404418143,0.125894532368176,-0.00898309914322813,0.0147241691924927,2.69,"""0"""
3,1,-1.35835406159823,-1.34016307473609,1.77320934263119,0.379779593034328,-0.503198133318193,1.80049938079263,0.791460956450422,0.247675786588991,-1.51465432260583,...,0.247998153469754,0.771679401917229,0.909412262347719,-0.689280956490685,-0.327641833735251,-0.139096571514147,-0.0553527940384261,-0.0597518405929204,378.66,"""0"""
4,1,-0.966271711572087,-0.185226008082898,1.79299333957872,-0.863291275036453,-0.0103088796030823,1.24720316752486,0.23760893977178,0.377435874652262,-1.38702406270197,...,-0.108300452035545,0.00527359678253453,-0.190320518742841,-1.17557533186321,0.647376034602038,-0.221928844458407,0.0627228487293033,0.0614576285006353,123.5,"""0"""
5,2,-1.15823309349523,0.877736754848451,1.548717846511,0.403033933955121,-0.407193377311653,0.0959214624684256,0.592940745385545,-0.270532677192282,0.817739308235294,...,-0.00943069713232919,0.79827849458971,-0.137458079619063,0.141266983824769,-0.206009587619756,0.502292224181569,0.219422229513348,0.215153147499206,69.99,"""0"""


In [6]:
#df.tail()
df = df.replace('"0"', '0')
df = df.replace('"1"', '1')

df = df.apply(pd.to_numeric, errors='ignore') # convert all columns of DataFrame 
df["V22"] = pd.to_numeric(df["V22"], errors='coerce')
df["Class"] = pd.to_numeric(df["Class"], errors='coerce')
df["Class"] = df["Class"].fillna(0)
df["Class"]=df['Class'].astype(int) 
#df.tail()
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 274826 entries, 1 to 274826
Data columns (total 31 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   Time    274826 non-null  float64
 1   V1      274826 non-null  float64
 2   V2      274826 non-null  float64
 3   V3      274826 non-null  float64
 4   V4      274826 non-null  float64
 5   V5      274826 non-null  float64
 6   V6      274826 non-null  float64
 7   V7      274826 non-null  float64
 8   V8      274826 non-null  float64
 9   V9      274826 non-null  float64
 10  V10     274826 non-null  float64
 11  V11     274826 non-null  float64
 12  V12     274826 non-null  float64
 13  V13     274826 non-null  float64
 14  V14     274826 non-null  float64
 15  V15     274826 non-null  float64
 16  V16     274825 non-null  float64
 17  V17     274825 non-null  float64
 18  V18     274825 non-null  float64
 19  V19     274825 non-null  float64
 20  V20     274825 non-null  float64
 21  V21     27

In [8]:
df.dropna(axis=0, how='any', thresh=None, subset=None, inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 274823 entries, 1 to 274826
Data columns (total 31 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   Time    274823 non-null  float64
 1   V1      274823 non-null  float64
 2   V2      274823 non-null  float64
 3   V3      274823 non-null  float64
 4   V4      274823 non-null  float64
 5   V5      274823 non-null  float64
 6   V6      274823 non-null  float64
 7   V7      274823 non-null  float64
 8   V8      274823 non-null  float64
 9   V9      274823 non-null  float64
 10  V10     274823 non-null  float64
 11  V11     274823 non-null  float64
 12  V12     274823 non-null  float64
 13  V13     274823 non-null  float64
 14  V14     274823 non-null  float64
 15  V15     274823 non-null  float64
 16  V16     274823 non-null  float64
 17  V17     274823 non-null  float64
 18  V18     274823 non-null  float64
 19  V19     274823 non-null  float64
 20  V20     274823 non-null  float64
 21  V21     27

In [10]:
df['Class'].value_counts()

0    274385
1       438
Name: Class, dtype: int64

In [11]:
legal= df[df.Class == 0]
fraud= df[df.Class == 1]

In [15]:
legal.shape
fraud.shape

(438, 31)

In [16]:
legal.describe()
fraud.describe()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
count,438.000000,438.000000,438.000000,438.000000,438.000000,438.000000,438.000000,438.000000,438.000000,438.000000,...,438.000000,438.000000,438.000000,438.000000,438.000000,438.000000,438.000000,438.000000,438.000000,438.0
mean,89136.139269,-4.768012,3.471939,-6.708899,4.249066,-3.168272,-1.316159,-5.419271,0.512529,-2.525094,...,0.789311,0.010541,-0.014364,-0.060176,0.063177,0.015473,0.090174,0.090655,126.875068,1.0
std,43877.546789,6.876980,4.425682,7.021080,2.680780,5.536087,1.897766,7.281668,7.173783,2.473596,...,4.063281,1.567493,1.365425,0.490757,0.721082,0.470996,1.363364,0.505873,249.346254,0.0
min,406.000000,-30.552380,-8.402154,-31.103685,-1.313275,-22.105532,-6.406267,-43.557242,-41.044261,-13.434066,...,-22.797604,-8.887017,-15.981649,-2.028024,-3.536716,-1.152671,-7.263482,-1.796363,0.000000,1.0
25%,50165.250000,-6.153202,1.130895,-8.213089,2.202591,-4.798396,-2.320769,-7.083026,-0.219446,-3.766028,...,0.098377,-0.540559,-0.330698,-0.398131,-0.292297,-0.265880,-0.036760,-0.097935,1.000000,1.0
50%,85285.000000,-2.319574,2.503879,-4.868289,3.953942,-1.379902,-1.281398,-2.873809,0.621508,-2.061335,...,0.614595,0.042493,-0.080533,-0.022755,0.103651,-0.043405,0.347954,0.141057,19.000000,1.0
75%,131820.500000,-0.434941,4.594065,-2.086762,6.131169,0.249133,-0.357445,-0.900824,1.743587,-0.795447,...,1.263300,0.623456,0.237815,0.323417,0.454747,0.285320,0.772292,0.370868,112.172500,1.0
max,170348.000000,2.132386,22.057729,2.250210,12.114672,11.095089,6.474115,5.802537,20.007208,3.353525,...,27.202839,8.361985,5.466230,1.091435,2.208209,2.745261,2.352333,1.779364,2125.870000,1.0


In [18]:
df.groupby('Class').mean()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount
Class,,,,,,,,,,,,,,,,,,,,,
0,97781.790962,0.014765,-0.015537,-0.020205,-0.021999,0.010674,-0.003641,0.018774,-0.001029,-0.045496,...,-0.000872,0.002354,0.006798,0.000986,-0.000047,-0.004673,-0.002013,-0.000149,-0.000215,89.194784
1,89136.139269,-4.768012,3.471939,-6.708899,4.249066,-3.168272,-1.316159,-5.419271,0.512529,-2.525094,...,0.365357,0.789311,0.010541,-0.014364,-0.060176,0.063177,0.015473,0.090174,0.090655,126.875068


In [19]:
#undersampling
legal_un_samp=legal.sample(n=438)
legal_un_samp.shape

(438, 31)

In [21]:
df_new=pd.concat([legal_un_samp, fraud], axis=0)
df_new.head()


,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
242155,155665.0,1.997284,-0.311574,-0.474082,0.327226,-0.410321,-0.267416,-0.508063,0.013772,1.279221,...,-0.192415,-0.447086,0.391653,0.615794,-0.395676,-0.651232,0.029343,-0.024376,10.00,0
254074,161242.0,1.683341,-0.495467,-1.974964,0.474104,0.211263,-0.665393,0.315280,-0.227438,0.946632,...,-0.141423,-0.564007,0.050130,0.505230,-0.097478,-0.132187,-0.040330,0.016131,190.06,0
232861,151679.0,-2.232867,0.875918,-2.062556,0.030441,0.271670,-1.115561,-0.116935,0.833181,-0.482509,...,0.336790,1.214829,0.603319,-0.436380,-0.111657,-0.166195,-0.816347,0.097138,30.00,0
75795,60935.0,1.001006,0.028864,0.165051,1.147460,-0.133374,-0.446199,0.318740,-0.155458,-0.123339,...,0.124886,0.235046,-0.168048,0.108107,0.596737,-0.311789,0.014492,0.036311,108.19,0
147432,109861.0,2.083319,0.143407,-1.331357,0.347331,0.438509,-0.711518,0.120913,-0.397370,1.896957,...,0.124226,0.807512,0.008112,0.591979,0.360152,-0.484629,-0.013816,-0.052833,1.00,0


In [22]:
df_new.groupby('Class').mean()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount
Class,,,,,,,,,,,,,,,,,,,,,
0,101424.068493,-0.009956,-0.087377,-0.239442,-0.025409,0.018030,-0.032024,-0.000842,0.056115,-0.023814,...,0.038483,0.070236,0.010091,0.017528,0.028627,0.012811,-0.010187,-0.012887,-0.004990,102.087968
1,89136.139269,-4.768012,3.471939,-6.708899,4.249066,-3.168272,-1.316159,-5.419271,0.512529,-2.525094,...,0.365357,0.789311,0.010541,-0.014364,-0.060176,0.063177,0.015473,0.090174,0.090655,126.875068


In [23]:
 df_new['Class'].value_counts()

0    438
1    438
Name: Class, dtype: int64

In [28]:
X=df_new.drop(columns='Class', axis=1)

Y=df_new['Class']
X
Y

242155    0
254074    0
232861    0
75795     0
147432    0
         ..
269883    1
270163    1
270169    1
271164    1
271694    1
Name: Class, Length: 876, dtype: int64

In [30]:
X_train, X_test, Y_train, Y_test= train_test_split(X,Y, test_size=.2, stratify=Y, random_state=2 )
Y_test


33644     1
83444     1
139607    1
62869     0
202664    1
         ..
130806    1
66629     1
231088    0
44260     0
233719    1
Name: Class, Length: 176, dtype: int64

In [34]:
model= LogisticRegression()
model.fit(X_train, Y_train)

LogisticRegression()

In [40]:
Y_train_pred=model.predict(X_train)
train_accuracy=accuracy_score(Y_train_pred, Y_train)
print('Accuracy on train =',train_accuracy)

Accuracy on train = 0.9328571428571428


In [41]:
Y_test_pred=model.predict(X_test)
test_accuracy=accuracy_score(Y_test_pred, Y_test)
print('Accuracy on test =',test_accuracy)

Accuracy on test = 0.9659090909090909
